# Module 2: Advanced Prompting Techniques

This module introduces you to additional prompting techniques that can improve the performance of your LLM application. This includes prompt engineering techniques as well as higher level code frameworks that can help build a system that is consistent and can be integrated into a larger codebase or application. We will end by introducing a modern LLM framework (Dspy) that abstracts away a lot of the baseline API configuration, and has higher level functionality like the ability to optimize your prompts for you.

In [1]:
pip install -r requirements1.txt

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import boto3
import base64
import os
from typing import List

import pydantic
from openai import OpenAI

In [ ]:
# Add the key for the AI Course below
 
os.environ["OPENAI_API_KEY"] = ""
OpenAI.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
# Confirm API connection works
client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "What is the best programming language for LLMs?"
        }
    ]
)

print(completion.choices[0].message.content)

In [ ]:
# List out all of the available models
# Chat completions API compatibility: https://platform.openai.com/docs/models#model-endpoint-compatibility
models_list = client.models.list().data
for model in models_list:
    print(model.id)

## Prompting 'Engineering' Techniques

![ontology](PA-ontology.png)

There are a lot of techniques that you can leverage when writing out prompts to elicit expected results. This paper has a pretty comprehensive review of techniques and when to use them: [link to paper](https://arxiv.org/pdf/2406.06608)

In [ ]:
def single_prompt_call(prompt, model='gpt-4o-mini'):
    completion = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": prompt
        }
        ]
    )
    return completion

In [ ]:
# Try a basic question without any examples
prompt = """Analyze the sentiment of this patient feedback:
‘Oh, great! Another drug that claims to work wonders but just emptied my wallet.'"""

response = single_prompt_call(prompt)
print(response.choices[0].message.content)

In [ ]:
# Try with few-shot prompting
prompt = """Here are examples of sentiment analysis for patient feedback:
	1.	Feedback: Drug X worked wonders for my migraines. Im so grateful!
Sentiment: Positive
	2.	Feedback: I had severe side effects with Drug Y and had to stop taking it.
Sentiment: Negative
	3.	Feedback: Drug Z was okay—it helped, but not as much as I hoped.
Sentiment: Mixed

Now, analyze the sentiment of this patient feedback:
Oh, great! Another drug that claims to work wonders but just emptied my wallet."""

response = single_prompt_call(prompt)
print(response.choices[0].message.content)

## Prompt Chaining

This is the technique primarily used when building out chatbots to preserve the history of previous messages as relevant context

In [ ]:
def call_with_messages(messages, model='gpt-4o-mini'):
    completion = client.chat.completions.create(
    model=model,
    messages=messages
    )
    return completion

In [ ]:
# Original message
messages=[
        {"role": "system", 
         "content": "You are a helpful assistant."},
        {
        "role": "user",
        "content": "What is the python programming language?"
        }
        ]

response = call_with_messages(messages)
print(response.choices[0].message.content)

In [ ]:
# Now we can append this answer to our messages list
messages.append({
    "role": "assistant",
    "content": response.choices[0].message.content
})
print(messages)

In [ ]:
# Lets add a follow-up question that refers to what was originally asked
prompt = "Tell me more"
messages.append({
    "role": "user",
    "content": prompt
})

response = call_with_messages(messages)
print(response.choices[0].message.content)

## Chain of Thought

![CoT](PA-CoT.png)

[Arxiv Paper Link](https://arxiv.org/pdf/2201.11903)

In [ ]:
text_to_extract_translation = """¡Preparar café Cold Brew es un proceso sencillo y refrescante!
Todo lo que necesitas son granos de café molido grueso y agua fría.
Comienza añadiendo el café molido a un recipiente o jarra grande.
Luego, vierte agua fría, asegurándote de que todos los granos de café
estén completamente sumergidos.
Remueve la mezcla suavemente para garantizar una saturación uniforme.
Cubre el recipiente y déjalo en remojo en el refrigerador durante al
menos 12 a 24 horas, dependiendo de la fuerza deseada."""

In [ ]:
baseline_prompt = f"""
Question: Give me a numbered list of all coffee-related words in English from the text below:

Text: {text_to_extract_translation}

Answer:
"""
baseline_response = single_prompt_call(baseline_prompt)
print(baseline_response.choices[0].message.content)

Did it give the list in english? This didn't follow the instructions well because it needs to do a few steps of reasoning before outputting the response. 

Lets try prompting it to break it into separate steps with an example (Chain of thought)

In [ ]:
# With Chain of Thought

CoT_prompt = f"""
Question: Give me a numbered list of all tennis-related words in English from the text below:

Text: Andre Agassi, una leyenda del tenis con ocho títulos de Grand Slam, fue celebrado por su poderoso juego de fondo y sus implacables devoluciones. 
Más allá de la corte, defendió la educación y fundó la Fundación Andre Agassi para jóvenes desfavorecidos. 
Sus memorias, Open, revelan su viaje de resiliencia, pasión y reinvención, inspirando a innumerables personas en todo el mundo.

Answer: The spanish words that are related to tennis are: tenis, juego de fondo, devoluciones. These words in english are: tennis, baseline, returns


Question: Give me a numbered list of all coffee-related words in English from the text below:

Text: {text_to_extract_translation}

Answer:
"""
CoT_response = single_prompt_call(CoT_prompt)
print(CoT_response.choices[0].message.content)

In [ ]:
# Come up with another text to test whether the chain of thought prompt gives an accurate answer

## Tree of Thought
![ToT](PA-ToT.png)

[Tree of thought Arxiv Paper](https://arxiv.org/pdf/2305.10601)

- The premise here is to generate a tree of expanding possible intermediate steps to fulfill the overall goal
- Bread-first-search (BFS) or Depth-first-search (DFS) can be used to traverse the tree of steps until a valid outcome is identified


We won't be coding this example

A task used in Tree of Thought was the mathematical reasoning challenge: Game of 24
- Given an input of 4 integers
- Use the 4 integers with any combination of basic arithmetic operations (+-*/) to obtain 24

![Game of 24](PA-ToT24.png)

## Multimodal Models

Multimodal models combine the ability of transformers to understand data across multiple modalities. The most common forms combine the language modality with images and video. This will be a demonstration of how to use images as inputs to multimodal models.

Documentation on this API: https://platform.openai.com/docs/guides/vision

Lets test this with a chart of EPCORE results:
![DOR_Chart](epcore_DOR.png)

In [ ]:
# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [ ]:
# Define the image file path
image_path = "epcore_DOR.png"

# Getting the base64 string
base64_image = encode_image(image_path)

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "What is in this image?",
        },
        {
          "type": "image_url",
          "image_url": {
            "url":  f"data:image/png;base64,{base64_image}"
          },
        },
      ],
    }
  ],
)

print(response.choices[0].message.content)


### Now lets try asking it questions and see how it performs

In [ ]:
def ask_image_questions(filepath, prompt, model="gpt-4o-mini"):
    base64_image = encode_image(filepath)

    response = client.chat.completions.create(
    model=model,
    messages=[
        {
        "role": "user",
        "content": [
            {
            "type": "text",
            "text": prompt,
            },
            {
            "type": "image_url",
            "image_url": {
                "url":  f"data:image/png;base64,{base64_image}"
            },
            },
        ],
        }
    ],
    )
    return response

In [ ]:
prompt = "What is the duration of response for 15th month mark?"
response = ask_image_questions(image_path, prompt)
print(response.choices[0].message.content)

In [ ]:
prompt = "How many subjects are at risk at the 15th month mark?"
response = ask_image_questions(image_path, prompt)
print(response.choices[0].message.content)

In [ ]:
### Try using another model

### Lets try another example with another chart type
![ORR_Chart](epcore_ORR.png)


In [ ]:
prompt = "What was the ORR for patients without Prior CAR-T experience?"
response = ask_image_questions('epcore_ORR.png', prompt, model="gpt-4o-mini")
print(response.choices[0].message.content)

## Structured Outputs

There are many instances when you want the LLM to provide responses in a specific format. This is most important when you want to build out more sophisticated systems, with subsequent calls, or if you simply need the output of the LLM to fit into a specific data structure. Structured outputs allows you to specify this when calling the API.

This is where Pydantic comes into play. It is a way to programmatically describe the data structure you want as an output.

Lets try to make the LLM extract information from a clinical trial protocol.

In [ ]:
trial_description = """The drug that will be investigated in the study is GEN1053. GEN1053 is an antibody designed to (re)activate and increase antitumor immunity.

Since this is the first study of GEN1053 in humans, the main purpose is to evaluate safety. Besides safety, the study will determine the recommended GEN1053 dose to be tested in a larger group of participants and assess preliminary clinical activity of GEN1053.

GEN1053 will be studied in a broad group of cancer patients, having different kinds of solid tumors. All participants will get GEN1053. The study consists of two parts: Part 1 tests increasing doses of GEN1053 ("escalation"), followed by Part 2 which tests the recommended phase 2 dose GEN1053 dose from Part 1 ("expansion").

The trial is a First in Human open-label, multicenter, multinational safety trial in participants with non-central nervous system (non-CNS) metastatic or advanced malignant solid tumors for whom there is no available standard therapy likely to confer clinical benefit, evaluating the safety, tolerability, preliminary antitumor activity, pharmacokinetics (PK), pharmacodynamics (PD), and immunogenicity of GEN1053.

The trial will be conducted as follows:

The Dose Escalation part (Part 1) will explore the safety of escalating doses of GEN1053 as monotherapy (phase 1)
The Expansion part (Part 2) is planned to provide additional safety and initial antitumor activity information of the Recommended Phase 2 dose (RP2D) for GEN1053 monotherapy in selected tumor indications, as well as more detailed data related to the mode of action (MoA)."""

baseline_study_prompt = f"""Extract the drug name and how many parts there are of the trial from the following clinical trial description:
{trial_description}"""

In [ ]:
baseline_study_response = single_prompt_call(baseline_study_prompt)
print(baseline_study_response.choices[0].message.content)

Now what if you wanted to parse these two responses out into an input for a database? 

Regex statement to extract the two answers out? (if you dont know how to create a regex pattern, try ChatGPT to help you)

What if there are extraneous characters?

In [ ]:
formatted_study_prompt = f"""Extract the drug name and how many parts there are of the trial from the following clinical trial description:
{trial_description}

Format your response like the following:
drug_name: drug name from clinical trial description
parts: count of parts of the study
"""

formatted_study_response = single_prompt_call(formatted_study_prompt)
print(formatted_study_response.choices[0].message.content)

Now you can parse it out easier, but what if you need to ensure the parts is an integer because you need to programmatically process the output differently depending on the answer?

How do you make sure the output conforms to your expectations so no errors occur when running your script?

In [ ]:
# Create this "pattern" by creating a class inheriting from the pydantic BaseModel class
from pydantic import BaseModel

class StudyOutput(BaseModel):
    drug: str
    parts: int

In [ ]:
study_prompt = f"""Extract the drug name and how many parts there are of the trial from the following clinical trial description:
{trial_description}"""

response = client.beta.chat.completions.parse(
  model="gpt-4o-mini",
  messages=[
      {"role": "system", "content": "You are a helpful assistant."},
      {"role": "user", "content": study_prompt},
  ],
  response_format=StudyOutput,
)

response_object = response.choices[0].message.parsed
response_object

In [ ]:
# Now create a pydantic class to extract a list of trial part descriptions

# Intro to DSPY

A framework for programming with LLMs

In [ ]:
import dspy

oa_model = dspy.OpenAI(model='gpt-4o-mini', max_tokens=250)
dspy.settings.configure(lm=oa_model)

Signatures are a way of functionalizing prompts. Create a signature with the class argstring being the prompt instructions. Inputs and outputs are defined with `dspy.InputField` and `dspy.OutputField` objects

Similar to how you can define the expected output format with the openai chat completions api, you can define the structure of the outputs with DSPY signatures as well

In [ ]:
class CTParse(dspy.Signature):
    """
    Extract the drug name from the following clinical trial description.
    """
    ct_description: str = dspy.InputField(desc="Clinical Trial description")
    drug: str = dspy.OutputField(desc="The drug name")

In [ ]:
ct_parser = dspy.ChainOfThought(CTParse)

parse_result = ct_parser(ct_description=trial_description)
parse_result

In [ ]:
# Now update the dspy signature to output the # of parts of the trial as well

## Extra Credit: Prompt Optimization with DSPY

In [ ]:
# from dspy.datasets import HotPotQA

# dataset = HotPotQA(train_seed=1, train_size=20, eval_seed=2023, dev_size=50, test_size=0)

# trainset, devset = dataset.train, dataset.dev

In [ ]:
# Loading in the dataset from file
import pickle

with open("hotpotqa_train.pkl", "rb") as f:
    trainset = pickle.load(f)

with open("hotpotqa_dev.pkl", "rb") as f:
    devset = pickle.load(f)

In [ ]:
trainset[0]

In [ ]:
class CoTSignature(dspy.Signature):
    """Answer the question and give the reasoning for the same."""

    question = dspy.InputField(desc="question about something")
    reasoning = dspy.OutputField(desc="reasoning for the answer")
    answer = dspy.OutputField(desc="often between 1 and 5 words")

class CoTPipeline(dspy.Module):
    def __init__(self):
        super().__init__()

        self.signature = CoTSignature
        self.predictor = dspy.ChainOfThought(self.signature)

    def forward(self, question):
        result = self.predictor(question=question)
        return dspy.Prediction(
            answer=result.answer,
            reasoning=result.reasoning,
        )

In [ ]:
test_pipe = CoTPipeline()
test_out = test_pipe('give me an answer')

In [ ]:
from dspy.evaluate import Evaluate

def validate_context_and_answer(example, pred, trace=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    return answer_EM

NUM_THREADS = 5
evaluate = Evaluate(devset=devset, metric=validate_context_and_answer, num_threads=NUM_THREADS, display_progress=True, display_table=False)

In [ ]:
cot_baseline = CoTPipeline()

devset_with_input = [dspy.Example({"question": r["question"], "answer": r["answer"]}).with_inputs("question") for r in devset]
evaluate(cot_baseline, devset=devset_with_input)

In [ ]:
from dspy.teleprompt import COPRO

teleprompter = COPRO(
    metric=validate_context_and_answer,
    verbose=True,
)

In [ ]:
kwargs = dict(num_threads=64, display_progress=True, display_table=0) # Used in Evaluate class in the optimization process

compiled_prompt_opt = teleprompter.compile(cot_baseline, trainset=devset_with_input, eval_kwargs=kwargs)

In [ ]:
# Print out the updated signature after optimization

compiled_prompt_opt

In [ ]:
# Make the necessary changes to the signature to match the optimized prompts

## Extra Credit: Implement Tree-of-Thought using structured outputs